In [1]:
from distributed import Client
import os
#import logging
os.environ["MODIN_ENGINE"] = "dask"
client = Client(silence_logs=50)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 224,Total memory: 503.61 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32801,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 224
Started: Just now,Total memory: 503.61 GiB
Comm: tcp://127.0.0.1:34965,Total threads: 14
Dashboard: http://127.0.0.1:45911/status,Memory: 31.48 GiB
Nanny: tcp://127.0.0.1:41365,


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import tensorflow as tf
import intel_extension_for_tensorflow as itex
import modin.pandas as pd
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, set_seed
from datasets import Dataset
from transformers import create_optimizer
import random
import re
import string
import os

2023-10-20 00:01:48.680930: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-20 00:01:48.685735: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 00:01:48.782579: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 00:01:48.784633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 00:01:54.711935: W tensorflow/compiler/tf2t

In [4]:
SEED = 42
set_seed = SEED
tf.keras.utils.set_random_seed(42)
random.seed(SEED)

In [5]:
BASE_DIR = '/home/u132668/'

In [6]:
train = pd.read_csv(os.path.join(BASE_DIR, 'data/train.csv'))

In [7]:
train.dropna(subset = ['span_text'], inplace=True)
train.drop(train[train['Answer'] == 'unknown'].index, inplace=True)

In [8]:
train['Question'] = train['Question'].apply(lambda x: x.lower().strip())
train['span_text'] = train['span_text'].apply(lambda x: x.lower().strip().strip(string.punctuation))
train['Answer'] = train['Answer'].apply(lambda x: x.lower().strip().strip(string.punctuation))

In [9]:
train_tf = train[(train['Answer'] == 'true') | (train['Answer'] == 'false')].copy()
train_yn = train[(train['Answer'] == 'yes') | (train['Answer'] == 'no')].copy()

# True/False Training:

In [10]:
train_tf = train_tf._to_pandas()

In [11]:
train_tf.rename(columns = {'Answer':'labels'}, inplace=True)

In [12]:
train_tf['labels'].replace('false', 0, inplace=True)
train_tf['labels'].replace('true', 1, inplace=True)

In [13]:
id2label_tf = {0: "false", 1: "true"}
label2id_tf = {"false": 0, "true": 1}

In [14]:
model_checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label_tf, label2id=label2id_tf)

2023-10-20 00:02:33.244413: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform XPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 00:02:33.244554: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:XPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: XPU, pci bus id: <undefined>)
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [16]:
dataset = Dataset.from_pandas(train_tf)

In [17]:
dataset = dataset.train_test_split(test_size=0.2, seed=SEED , shuffle=True)

In [18]:
pad_on_right = True
max_length = 128
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["Question" if pad_on_right else "span_text"],
        examples["span_text" if pad_on_right else "Question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        padding="max_length",
    )
    return tokenized_examples

In [19]:
tokenized_data = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)
train_tokenized_data = tokenized_data['train'].add_column('labels', dataset['train']['labels'])
test_tokenized_data = tokenized_data['test'].add_column('labels', dataset['test']['labels'])

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [20]:
batch_size = 1
num_epochs = 3
total_train_steps = (len(tokenized_data["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=1e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
    #weight_decay_rate = 0.01
)

In [21]:
tf_train_set = train_tokenized_data.to_tf_dataset(
       columns=['input_ids', 'attention_mask', 'label'],
       batch_size=batch_size
    )
tf_validation_set = test_tokenized_data.to_tf_dataset(
       columns=['input_ids', 'attention_mask', 'label'],
       batch_size=batch_size
    )

In [22]:
model.compile(optimizer=optimizer, metrics=['accuracy'])

In [23]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

Epoch 1/3


2023-10-20 00:03:21.790372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


44/44 [==============================] - ETA: 0s - loss: 1.4394 - accuracy: 0.6364

2023-10-20 00:03:45.169724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


44/44 [==============================] - 56s 322ms/step - loss: 1.4394 - accuracy: 0.6364 - val_loss: 2.2391 - val_accuracy: 0.4167
Epoch 2/3
44/44 [==============================] - 8s 181ms/step - loss: 0.7689 - accuracy: 0.7500 - val_loss: 1.7260 - val_accuracy: 0.4167
Epoch 3/3
44/44 [==============================] - 9s 197ms/step - loss: 0.5438 - accuracy: 0.7955 - val_loss: 1.7204 - val_accuracy: 0.4167


In [24]:
model.save_pretrained(os.path.join(BASE_DIR,'saved_models/BoolQA/TrueFalseQA/v2'))
tokenizer.save_pretrained(os.path.join(BASE_DIR,'saved_models/BoolQA/TrueFalseQA/v2'))

('/home/u132668/saved_models/BoolQA/TrueFalseQA/v2/tokenizer_config.json',
 '/home/u132668/saved_models/BoolQA/TrueFalseQA/v2/special_tokens_map.json',
 '/home/u132668/saved_models/BoolQA/TrueFalseQA/v2/vocab.txt',
 '/home/u132668/saved_models/BoolQA/TrueFalseQA/v2/added_tokens.json',
 '/home/u132668/saved_models/BoolQA/TrueFalseQA/v2/tokenizer.json')

In [25]:
model.push_to_hub("WaRKiD/distilbert-base-uncased-finetuned-intel-llm-tf-dataset")
tokenizer.push_to_hub("WaRKiD/distilbert-base-uncased-finetuned-intel-llm-tf-dataset")

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/WaRKiD/distilbert-base-uncased-finetuned-intel-llm-tf-dataset/commit/76a94084d309f27fc6d51ff8de218b7cadb89f21', commit_message='Upload tokenizer', commit_description='', oid='76a94084d309f27fc6d51ff8de218b7cadb89f21', pr_url=None, pr_revision=None, pr_num=None)

# Yes/No Training:

In [27]:
train_yn = train_yn._to_pandas()

In [28]:
train_yn.rename(columns = {'Answer':'labels'}, inplace=True)

In [29]:
train_yn['labels'].replace('no', 0, inplace=True)
train_yn['labels'].replace('yes', 1, inplace=True)

In [30]:
id2label_yn = {0: "no", 1: "yes"}
label2id_yn = {"no": 0, "yes": 1}

In [31]:
model_checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label_yn, label2id=label2id_yn)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [33]:
dataset = Dataset.from_pandas(train_yn)

In [34]:
dataset = dataset.train_test_split(test_size=0.2, seed=SEED , shuffle=True)

In [35]:
pad_on_right = True
max_length = 128
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["Question" if pad_on_right else "span_text"],
        examples["span_text" if pad_on_right else "Question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        padding="max_length",
    )
    return tokenized_examples

In [36]:
tokenized_data = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)
train_tokenized_data = tokenized_data['train'].add_column('labels', dataset['train']['labels'])
test_tokenized_data = tokenized_data['test'].add_column('labels', dataset['test']['labels'])

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/1965 [00:00<?, ? examples/s]

In [37]:
batch_size = 8
num_epochs = 3
total_train_steps = (len(tokenized_data["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=1e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
    #weight_decay_rate = 0.01
)

In [38]:
tf_train_set = train_tokenized_data.to_tf_dataset(
       columns=['input_ids', 'attention_mask', 'label'],
       batch_size=batch_size
    )
tf_validation_set = test_tokenized_data.to_tf_dataset(
       columns=['input_ids', 'attention_mask', 'label'],
       batch_size=batch_size
    )

In [39]:
model.compile(optimizer=optimizer, metrics=['accuracy'])

In [40]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

Epoch 1/3


2023-10-20 00:11:31.377618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


983/983 [==============================] - ETA: 0s - loss: 0.6333 - accuracy: 0.6752

2023-10-20 00:14:51.887587: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


983/983 [==============================] - 250s 216ms/step - loss: 0.6333 - accuracy: 0.6752 - val_loss: 0.5191 - val_accuracy: 0.7486
Epoch 2/3
983/983 [==============================] - 203s 206ms/step - loss: 0.4562 - accuracy: 0.7870 - val_loss: 0.4849 - val_accuracy: 0.7898
Epoch 3/3
983/983 [==============================] - 206s 210ms/step - loss: 0.3401 - accuracy: 0.8595 - val_loss: 0.4899 - val_accuracy: 0.7858


In [41]:
model.save_pretrained(os.path.join(BASE_DIR,'saved_models/BoolQA/YesNoQA/v2'))
tokenizer.save_pretrained(os.path.join(BASE_DIR,'saved_models/BoolQA/YesNoQA/v2'))

('/home/u132668/saved_models/BoolQA/YesNoQA/v2/tokenizer_config.json',
 '/home/u132668/saved_models/BoolQA/YesNoQA/v2/special_tokens_map.json',
 '/home/u132668/saved_models/BoolQA/YesNoQA/v2/vocab.txt',
 '/home/u132668/saved_models/BoolQA/YesNoQA/v2/added_tokens.json',
 '/home/u132668/saved_models/BoolQA/YesNoQA/v2/tokenizer.json')

In [42]:
model.push_to_hub("WaRKiD/distilbert-base-uncased-finetuned-intel-llm-yn-dataset")
tokenizer.push_to_hub("WaRKiD/distilbert-base-uncased-finetuned-intel-llm-yn-dataset")

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/WaRKiD/distilbert-base-uncased-finetuned-intel-llm-yn-dataset/commit/52869c40a1d4e72cc5b9e6b3d42857a588aa131b', commit_message='Upload tokenizer', commit_description='', oid='52869c40a1d4e72cc5b9e6b3d42857a588aa131b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
train_tf_no_null = train_tf[train_tf['span_text'].isnull() == False]

In [ ]:
train_tf_no_null['span_text'] = train_tf_no_null['span_text'].apply(lambda x: x.lower().strip().strip(string.punctuation))

In [ ]:
train_tf_no_null['answer_start'] = train_tf_no_null.apply(lambda row: row['Story'].rfind(row['span_text']), axis=1)

In [ ]:
train_tf_no_null[train_tf_no_null['answer_start']==0]

In [ ]:
train_tf_no_null[train_tf_no_null['answer_start']==0].iloc[0]['Story']

In [ ]:
train_tf_no_null[train_tf_no_null['answer_start']==0].iloc[0]['Question']

In [ ]:
train_tf_no_null[train_tf_no_null['answer_start']==0]['span_text']

In [ ]:
train_tf_no_null[train_tf_no_null['answer_start']==0].iloc[0]['Answer']

In [ ]:
train_tf_no_null.iloc[0]['span_text']

In [ ]:
tokenizer_qa = AutoTokenizer.from_pretrained(PATH_QA, local_files_only=True)
model_qa = TFAutoModelForQuestionAnswering.from_pretrained(PATH_QA, local_files_only=True)

In [ ]:
train_tf = train[(train['Answer'] == 'true') | (train['Answer'] == 'false')].copy()
train_yn = train[(train['Answer'] == 'yes') | (train['Answer'] == 'no')].copy()

In [ ]:
print('Number of true/false questions: ' + str(len(train_tf)))
print('Number of yes/no questions: ' + str(len(train_yn)))

In [ ]:
verbs = set(['am','is','are','was','were','has','have','had','shall','will','can','should','would','could','must','may','might','do','does','did'])

In [ ]:
test = pd.read_csv(os.path.join(BASE_DIR, 'test.csv'))

In [ ]:
def check_boolq(question):
    first_word = question.split(' ')[0]
    if first_word in verbs:
        return 1
    if question.rfind('true or false')>=0:
        return 2
    return 0

In [ ]:
test['Question'] = test['Question'].apply(lambda x: x.lower().strip())
test['check_boolq'] = test['Question'].apply(check_boolq)

In [ ]:
from transformers import pipeline

In [ ]:
out_file = pd.read_csv('D:\\ML\\oneAPI Hackathon - The LLM Challenge\\test_submission.csv')

In [ ]:
out_file.iloc[test[test['check_boolq'] == 2].index]['Answer']

In [ ]:
test[test['check_boolq'] == 2]['Question'].head()

In [ ]:
model_tf_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
result_tf = model_tf_pipeline((test[test['check_boolq'] == 2]['Question']+ ' ' +out_file.iloc[test[test['check_boolq'] == 2].index]['Answer']).to_list())

In [ ]:
for i,j in enumerate(test[test['check_boolq'] == 2].index):
    out_file.iloc[j]['Answer'] = result_tf[i]['label']

In [ ]:
model_yn_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
result_yn = model_yn_pipeline((test[test['check_boolq'] == 1]['Question']+ ' ' +out_file.iloc[test[test['check_boolq'] == 1].index]['Answer']).to_list())

In [ ]:
for i,j in enumerate(test[test['check_boolq'] == 1].index):
    out_file.iloc[j]['Answer'] = result_yn[i]['label']

In [ ]:
out_file.to_csv('D:\\ML\\oneAPI Hackathon - The LLM Challenge\\out\\submission_v7.csv',index=False)